# 1.Loading data.

In [1]:
import pandas as pd
trainDf = pd.read_csv('Train.csv')
testDf = pd.read_csv('Test_A.csv')

In [2]:
trainDf.iloc[0:200][:]

,SessionId,Role,Text,HighRiskFlag
0,session0,坐席,诶您好PRD您是NAME先生是吧,0
1,session0,客户,诶是是,0
2,session0,坐席,还没的时候我会逾期了您今天app还款NUM块钱先处理NUM下好吗,0
3,session0,客户,okokok信息今天还尽量今天还掉啊,0
4,session0,坐席,NUM天了,0
...,...,...,...,...
195,session1,坐席,为啥子了,1
196,session1,坐席,这边就只能周日呀,1
197,session1,坐席,这地觉得你家里诶诶诶还了也就骚扰了就赶紧下来才才能还这钱是吧,1
198,session1,坐席,那你当时签合同时候上了款没下来啊干嘛,1


In [3]:
testDf.iloc[0:30][:]

,SessionId,Role,Text
0,session0,坐席,您好PRD
1,session0,坐席,我明天想NUM想吗
2,session0,坐席,不上那就
3,session1,客户,抱歉我现在无法接听您的电话本次呼叫将以PRD消息的方式通知我
4,session1,客户,感谢您的来电
5,session2,客户,喂
6,session2,坐席,嗯
7,session2,客户,喂
8,session2,坐席,NAME您好
9,session2,客户,诶


Analyse data.

In [16]:
class count(object):
#     Two parameters are presented as: dataframe for count, count session number only or session number risk flag number. 
    def __init__(self, df, session_only=True):
        self.df = df
        self.session_num = 0
        self.flag_0 = 0
        self.flag_1 = 0
        self.session_only = session_only
        
    def get_num(self):
        self.session_num += 1
        last_id = self.df.iloc[0][0]
        
        if self.session_only:
            for i in self.df.index:
                if self.df.iloc[i][0] == last_id:
                    continue
                else:
                    self.session_num += 1
                    last_id = self.df.iloc[i][0]
                    
            return self.session_num
        
        
        else:
            if self.df.iloc[0][3] == 0:
                self.flag_0 += 1
            else:
                self.flag_1 += 1
                
            for i in self.df.index:
                if self.df.iloc[i][0] == last_id:
                    continue
                else:
                    self.session_num += 1
                    last_id = self.df.iloc[i][0]
                    if self.df.iloc[i][3] == 0:
                        self.flag_0 += 1
                    else:
                        self.flag_1 += 1
                        
            return self.session_num, self.flag_0, self.flag_1
        
        

In [13]:
counter = count(testDf, session_only=True)
counter.get_num()

50069

In [17]:
counter1 = count(trainDf, session_only=False)
counter1.get_num()

ok


(135656, 134462, 1194)

# 2.Cut words and store it.

In [2]:
import re
import os
import jieba

In [11]:
def cut_word(df, flag):
    lastId = df.iloc[0][0]
    WordsL = []
    strTemp = ''
    wordsTemp = []
    flagL = []
    if flag: flagL.append(df.iloc[0][3])
    
    for i in df.index:
        words = ' '.join(jieba.cut(df.iloc[i][2])).split(' ')
        
        if df.iloc[i][0] == lastId:
            for word in words:
                strTemp += word + ' '
        else:
            wordsTemp.append(strTemp)
            WordsL.append(wordsTemp)
            if flag: flagL.append(df.iloc[i][3])
            
            lastId = df.iloc[i][0]
            wordsTemp = []
            strTemp = ''
            
            for word in words:
                strTemp += word + ' '
                
    wordsTemp.append(strTemp)
    WordsL.append(wordsTemp)
    
    return WordsL, flagL

In [12]:
trainWordsL, trainFlagL = cut_word(trainDf, True)

In [13]:
trainWordsL

[['诶 您好 PRD 您 是 NAME 先生 是 吧 诶 是 是 还 没 的 时候 我会 逾期 了 您 今天 app 还款 NUM 块钱 先 处理 NUM 下好 吗 okokok 信息 今天 还 尽量 今天 还 掉 啊 NUM 天 了 这边 的话 会上 传 征信 您 抓紧时间 再见 '],
 ['喂 您好 是 NAME 先生 是 吧 诶 先生 您 那个 PRD 的 NUM 块钱 还 没 还 啊 嗯 好 吧 刚才 不有 事儿 嘛 这 是 哪里 提前 你 有 本事 了 先生 这钱 还 不 还 啊 PRD 行 吧 还 呢 啊 对 呀 还 的 认证 是 什么 时候 还 那 要 有望 困难 你 可以 直说 啊 对 吧 我 这边 也 不是 不能 也 不是 说 不能 理解 你 也 怎么着 的 啊 呃 你 要 您 这 待会儿 您 那个 要是 不 说话 的话 你 这样 不 说话 的话 那 咱 这 电脑 就 自动 为 你 这个 合同 接通 不 说 就 直接 直接 视为 你 这个 不 愿意 还款 了 懂吗 你 看 下 有 什么 困难 你 就 直接 说 对 吧 有 什么 事儿 要 商量 什么 事儿 了 NUM 架 我 这儿 没有 意思 了 啊 NAME 各 吧 我 忘 了 喂 我 真是 服 了 你 了 你 天天 都 下 了 不到 NUM 块钱 儿 我 电话 好 的 你 不 说话 有用吗 就 就 该 你 现在 躲 在 电话号码 的话 我们 这边 又 找不着 你 你 不要 天天 海城区 抗战 路 嘛 咱们 国家 告 你 刚刚 说 就 就 在 添加 服 和平解决 不 可能 找 不到 的 你 要 什么 事儿 你 就 说 什么 事儿 了 懂吗 你 要是 说 你 现在 没钱 啊 或者说 其他 的 什么 事儿 你 就 直说 我 还 能 逼 着 你 怎么 怎么着 啊 我们 NUM 直给 您 打电话 不是 说怕 估计 想 了解 您 情况 看 你 这 钱 能 不能 还 呀 还 不了 跟 你 做 登记 啊 对 吧 他 给 你 天天 催 我 给 你 涨 那个 罚息 吃 然后 然后 给 你 上 征信 吗 为了 您好 以 下载 着 我 的 电话 号儿 话 也 不 多 NUM 点 是 吗 就 废 了 到时候 我们 给 您 办理 吗 打电话 然后 这个 时候 你 才 你 才 里边儿 给 我 了 是 吗 是 啊 喂 不是

In [14]:
trainFlagL

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [15]:
trainWordsDf = pd.DataFrame(trainWordsL,columns=['Cut'])
trainWordsDf['Flag'] = trainFlagL
trainWordsDf.to_csv('conv_train.csv', index=0)
trainWordsDf

,Cut,Flag
0,诶 您好 PRD 您 是 NAME 先生 是 吧 诶 是 是 还 没 的 时候 我会 逾期 ...,0
1,喂 您好 是 NAME 先生 是 吧 诶 先生 您 那个 PRD 的 NUM 块钱 还 没 ...,1
2,PRD NAME 先生 吗,0
3,PRD 的 请问 您 是 NAME 先生 是 吗 对 嗯 PRD 严重 拖欠 NUM 天 了...,0
4,您好 PRD 请问 是 NAME 先生 吗 武昌 再见,0
...,...,...
135651,喂 PRD 是 PRD 截行 吧 对 你 这个 PRD 了 NUM 块钱 啊 你 不是 说 ...,0
135652,嗯 诶 喂 您好 就 去 PRD 诶 已经 在 我 我 还 了 但是 每次 都 是 在 代扣...,0
135653,我 请问 是 NAME 先生 吗 啊 对 我 知道 你 是 没 他 跟 那个 PRD 嘛 我...,0
135654,诶 是 啊 嗯 喂 你好 黄海 到 什么 你好 什么 事 怎么 说 哦 我们 这边 是 这个...,0


In [24]:
testWordsL, testFlagL = cut_word(testDf, False)

In [25]:
testWordsL

[['您好 PRD 我 明天 想 NUM 想 吗 不上 那 就 '],
 ['抱歉 我 现在 无法 接听 您 的 电话 本次 呼叫 将 以 PRD 消息 的 方式 通知 我 感谢您 的 来电 '],
 ['喂 嗯 喂 NAME 您好 诶 NAME 女士 是 吧 嗯 对 NAME 女士 您好 现在 NUM 点 了 我 中国 儿子 之后 要 答应 你 了 这 段时间 没有 给 你 家里人 任何 NUM 个人 打电话 没 给 你 打电话 这样 我 这边 答应 你 了 你 这边 你 最起码 有 NUM 个 还款 态度 和 我 说 NUM 下好 吧 嗯 我 不 我 不是 跟 你 说 了 嘛 我 现在 实在 没有 用 了 我会 还 进去 了 对 吧 那 女士 您 现在 您 您 现在 是 NUM 分钱 没有 没有 嘛 嗯 嗯 今天 我 现在 没有 过 几天 我 有 了 我会 还 清楚 嗯 过 几天 有 了 那 女士 您 您 也 知道 对 吧 我 就是 说白了 这个 钱 您 欠 公司 也 不是 说 欠 块钱 我 和 您 就是 在 借 没什么 用 嗯 嗯 我 儿子 这边 好核 您 就是 聊 NUM 下 咱们 就是 小 NUM 个 折中 的 办法 把 这个 事情 解决 NUM 下 对 吧 我 也 就 好好 河 您 咱们 解决 这个 爸爸 我 也 不想 给 你家 人家 人 朋友 打电话 对 吧 说白了 给 他们 打电话 嗯 他们 有钱 还是 没 钱 的话 那 的 您 还 不 上 那 也 没用 呀 对 吧 对 那 您 到时候 他们 不会 打地 了 他们 他们 再 借个 借个 借 了 借 了 他们 也 不 可能 了 那 也 没用 是 啊 对 对 对 所以 说 对 呃 咱 就是 打 就是 咱们 就是 打电话 就是 我 和 当事人 吗 毕竟 您 是 欠债人 我 咱们 河 当事人 想想 想想 办法 然后 我 解决 这个 办法 嗯 我 嗯 嗯 NAME 女士 嗯 因为 这边 您 也 知道 咱们 通话 都 有 录音 的 我 说白了 我 也 是 给 人家 打工 的 嗯 到 这个 机关 您 说 您 过 几天 还个 因为 这通 录音 的话 也 不是 NUM 次 NUM 次 了 美 每次 都 过 几天 还 他 那个 公司 不 相信 嗯 嗯 知道 吗 不是 他 那个 公司 你 最起码 你 最近 有 NUM 个 

In [26]:
testFlagL

[]

In [27]:
testWordsDf = pd.DataFrame(testWordsL,columns=['Cut'])
testWordsDf.to_csv('conv_test.csv', index=0)
testWordsDf

,Cut
0,您好 PRD 我 明天 想 NUM 想 吗 不上 那 就
1,抱歉 我 现在 无法 接听 您 的 电话 本次 呼叫 将 以 PRD 消息 的 方式 通知 ...
2,喂 嗯 喂 NAME 您好 诶 NAME 女士 是 吧 嗯 对 NAME 女士 您好 现在 ...
3,喂 嗯 喂 嗯 诶 您好 您 这边 是 那个 NAME 的 朋友 是 吗 我 就是 成都 对...
4,诶 您好 PRD 您好 NAME 先生 是 吧 喂 您好 是 NAME 先生 吗 我 现在
...,...
50064,喂
50065,喂 您好 呃 请问 是 NAME 先生 吗 NAME 先生 您 这边 PRD 逾期 NUM ...
50066,诶 您好 PRD 是 王国 发现 是 吧 NAME 先生 PRD 贷款 逾期 了 不是 坏事...
50067,嗯 挂 啊 喂 你好 是 那个 和 NAME 的 爱人 那个 NAME 女士 对 吧 你们 ...


# 3.Using Tokenizer to encoding the word set.

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [13]:
max_words = 10000
max_len = 300
tok = Tokenizer(num_words=max_words)  ## The max word number is 5000
tok.fit_on_texts(trainWordsDf.Cut)

In [14]:
train_seq = tok.texts_to_sequences(trainWordsDf.Cut)
test_seq = tok.texts_to_sequences(testWordsDf.Cut)

In [15]:
train_seq_mat = sequence.pad_sequences(train_seq,maxlen=max_len)
test_seq_mat = sequence.pad_sequences(test_seq,maxlen=max_len)

In [16]:
## Store the trained Tokenizer and download.
import pickle
# saving
with open('tok.pickle', 'wb') as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tok.pickle', 'rb') as handle:
    tok = pickle.load(handle)

In [17]:
train_y = trainWordsDf.Flag.values